# CS295/395: Secure Distributed Computation
## In-Class Exercise, week of 10/03/2022

In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import numpy as np
import galois

from nacl.public import PrivateKey, Box, SealedBox

# GF = galois.GF(2**13 - 1)
GF_2 = galois.GF(2) # we work in the binary field this week!

# Library for circuits
from dataclasses import dataclass

@dataclass
class Gate:
    type: str
    in1: int
    in2: int
    out: int

@dataclass
class Circuit:
    inputs: any
    outputs: any
    gates: any
        
def print_circuit(c):
    print('inputs:', c.inputs)
    print('outputs:', c.outputs)
    print('gates:')
    for g in c.gates:
        print('  ', g)

## Party Class

In [ ]:
class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self):
        """Initialize the field size and dictionary to hold received messages."""
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

## Question 1

Describe the 1-out-of-2 *oblivious transfer* (OT) protocol. Reference Section 3.7 in Pragmatic MPC.

YOUR ANSWER HERE

## Question 2

Why is the oblivious transfer protocol secure against semi-honest adversaries? Why is it not secure against malicious adversaries?

YOUR ANSWER HERE

## Question 3

Implement 1-out-of-2 OT.

In [ ]:
class OT_Sender(Party):
    # x1 and x2 are the secrets
    def round1(self, x1, x2, receiver):
        self.x1 = x1
        self.x2 = x2
        self.receiver = receiver

    def round2(self):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round3(self):
        pass

class OT_Receiver(Party):
    def round1(self, b, sender):
        self.sender = sender
        self.b = b
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round2(self):
        pass
    
    def round3(self):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE
sender = OT_Sender()
receiver = OT_Receiver()

# Round 1
sender.round1(GF_2(0), GF_2(1), receiver)
receiver.round1(GF_2(1), sender)

# Round 2
sender.round2()
receiver.round2()

# Round 3
sender.round3()
output = receiver.round3()

print("Receiver's output:", output)
assert output == 1

## Question 4

Describe 1-out-of-4 OT.

YOUR ANSWER HERE

## Question 5

Describe a method for evaluating an `AND` gate using 1-out-of-4 OT on additive-secret-shared inputs.

YOUR ANSWER HERE

## Question 6

Implement the function $T_G$ that computes the truth table for an `AND` gate with input wires $i$ and $j$ based on input shares of P1 and P2 and output share $r$ for P1.

In [ ]:
# P1 holds shares s1_*
# P2 holds shares s2_*
def S(s1_i, s1_j, s2_i, s2_j):
    return (s1_i + s2_i) * (s1_j + s2_j)

# P1 holds shares s1_*
# P1 generates a random output share r
def T_G(r, s1_i, s1_j):
    # YOUR CODE HERE
    raise NotImplementedError()

T_G(GF_2(0), GF_2(1), GF_2(0))

In [ ]:
# TEST CASE
s1_i, s1_j, s2_i, s2_j = GF_2([0, 1, 1, 0])
row_num = 2 # because of the position of (1, 0) in the table computed by T_G
for _ in range(10): # try it 10 times, to account for randomness
    r = GF_2.Random()
    table = T_G(r, s1_i, s1_j)
    p2_share = table[row_num]
    output_result = r + p2_share
    assert output_result == GF_2(1)

## Question 7

Implement the `AND` gate protocol from above.

In [ ]:
class AND_P1(Party):
    # x1 and x2 are the secrets
    def round1(self, s1_i, s1_j, p2):
        self.s1_i = s1_i
        self.s1_j = s1_j
        self.p2 = p2

    def round2(self):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round3(self):
        return self.output

class AND_P2(Party):
    def round1(self, s2_i, s2_j, p1):
        self.p1 = p1
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round2(self):
        pass
    
    def round3(self):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE

for _ in range(5): # try it a few times
    p1 = AND_P1()
    p2 = AND_P2()
    s1_i, s1_j, s2_i, s2_j = GF_2([0, 1, 1, 0])

    # Round 1
    p1.round1(s1_i, s1_j, p2)
    p2.round1(s2_i, s2_j, p1)

    # Round 2
    p1.round2()
    p2.round2()

    # Round 3
    output_share1 = p1.round3()
    output_share2 = p2.round3()

    print("P1's output:", output_share1)
    print("P2's output:", output_share2)
    assert output_share1 + output_share2 == 1

## Question 8

Describe the GMW protocol for evaluating a binary circuit.

YOUR ANSWER HERE

## Question 9

Describe the main idea behind Yao's garbled circuit protocol.

YOUR ANSWER HERE

## Question 10

What are the primary advantages and disadvantages of Yao's garbled circuit protocol?

YOUR ANSWER HERE